In [1]:
"""
Infers α in closed form
"""
function infer_alpha!(dtKtu, ddtKtu, Kuu, I, θ_opt; 
    η_RKHS=1e1)
    
    C, α, u, N, M, D, J = name_params(θ_opt)
    
    average_spike_num = convert(Float64,sum(vcat(I[:]...)))/N; # Spikes per trial, average
    
    for n = 1:N   
        
        # Compute Cext
        Itot = convert(Integer, sum(I[n]))
        Jtot = convert(Integer, sum(J))
        Cext = zeros(Itot,Jtot)
        for m = 1:M
            m_inds = (m>1?cumsum(I[n])[m-1]+1:1):cumsum(I[n])[m];
            for d = 1:D
                d_inds = (d>1?cumsum(J)[d-1]+1:1):cumsum(J)[d];
                Cext[m_inds, d_inds] = C[m,d]
            end
        end
        
        dCK = Cext.*dtKtu[n] # Itot x Jtot
        ddCK = Cext.*ddtKtu[n] # Itot x Jtot
        
        # Solve for α on trial n
        α_opt = -(0.5*(dCK' * dCK) + η_RKHS * average_spike_num * Kuu[n])\(sum(ddCK,1)')
        
        # Sort it into actual α
        for d = 1:D
            d_inds = (d>1?cumsum(J)[d-1]+1:1):cumsum(J)[d];
            θ_opt.α[n,d] = α_opt[d_inds]
        end
    end
end


function learn_C!(dtKtu, ddtKtu, Kuu, I, θ_opt; 
    η_Cm=1e1)
    
    C, α, u, N, M, D, J = name_params(θ_opt)
    
    average_spike_num = convert(Float64,sum(vcat(I[:]...)))/M; # Spikes per neuron, average

    # Solve for each Cm separately.
    for m = 1:M
        
        # Build the all_spikes of neuron n x appropriate latent functions matrices
        In = Array(Integer, N)
        for n = 1:N In[n] = I[n][m] end
        Itot = sum(In)
        dαK = zeros(Itot, D)
        ddαK = zeros(Itot, D)
        
        for n = 1:N
            n_inds = (n>1?cumsum(In)[n-1]+1:1):cumsum(In)[n]; # Spikes of neuron m on trial n in currently built matrix
            m_inds = (m>1?cumsum(I[n])[m-1]+1:1):cumsum(I[n])[m]; # Spikes of neuron m on trial n in original kernel matrix
            for d = 1:D
                d_inds = (d>1?cumsum(J)[d-1]+1:1):cumsum(J)[d]; # Inducing points belonging to kernel d
                dαK[n_inds, d] = dtKtu[n][m_inds, d_inds]*α[n,d] # First derivative
                ddαK[n_inds, d] = ddtKtu[n][m_inds, d_inds]*α[n,d] # Second derivative
            end
        end
        
        #sddαK = (sum(ddαK,1)')
        #dαK2 = (dαK' * dαK)
        #Cm_opt_func(Cm_opt) = (Cm_opt'*dαK2*Cm_opt + sddαK*Cm_opt + η_Cm * (norm(Cm_opt) + norm(Cm_opt, Inf)))[1]
        #res = Optim.optimize(Cm_opt_func, C[m,:]')     
        #Cm_opt = Optim.minimizer(res)
        
        # Solve closed for C[m,:] with only quadratic penaly
        Cm_opt = - ( 0.5*(dαK' * dαK) + η_Cm * average_spike_num * eye(D)) \ (sum(ddαK,1)')
        
        C[m,:] = Cm_opt'
        
        # If solution starts to diverge, reset
        if maximum(Cm_opt) > 5
            C[m,:] = ones(size(C[m,:]))
        end
    end
    
end


"""
Optimises θ_opt in place
"""
function optimise!(data::AbstractArray, θ_opt::Union{PoissonProcessParams, AbstractArray}, KernList::AbstractArray;
    num_iters=1, η_RKHS=1e1, η_Cm = 1e1)
    
    # Compute the required kernel matrices (between inducing points and data points)
    @time Ktu, dtKtu, ddtKtu, Kuu, I = build_kernels(data, θ_opt, KernList)
    println("Kernels have been built")
    println("")
    
    println("Starting optimisation, initial cost is $(Cost(Ktu, dtKtu, ddtKtu, Kuu, I, θ_opt, KernList; η_RKHS=η_RKHS, η_Cm = η_Cm))")
    println("")
    
    for iters = 1:num_iters
        println("Starting iter $(iters)")
        @time infer_alpha!(dtKtu, ddtKtu, Kuu, I, θ_opt, η_RKHS=η_RKHS)
        println("Alphas inferred, cost is $(Cost(Ktu, dtKtu, ddtKtu, Kuu, I, θ_opt, KernList; η_RKHS=η_RKHS, η_Cm = η_Cm))")
        @time learn_C!(dtKtu, ddtKtu, Kuu, I, θ_opt, η_Cm=η_Cm)
        println("C learned, cost is $(Cost(Ktu, dtKtu, ddtKtu, Kuu, I, θ_opt, KernList; η_RKHS=η_RKHS, η_Cm = η_Cm))")
        println("")
    end
    
    return Ktu, dtKtu, ddtKtu, Kuu, I
end

LoadError: LoadError: UndefVarError: PoissonProcessParams not defined
while loading In[1], in expression starting on line 433